In [17]:
import json
import requests
import urllib.request
import time
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
df = pd.DataFrame()
index = 0
last_date = ''

# 수정 변수들
coinName = "BTC"
to_date = '2021-06-21'
num = 30
scale = '1'
if num == 0:
    type = 'days'
else:
    type = 'minutes/{}'.format(num)


def get_coin_data_url(coinName, type, scale, cnt=400) :
    addr = 'https://crix-api-endpoint.upbit.com/v1/crix/candles/'
    if type == 'minutes' :
        basic_url = addr + type + '/' + scale + '?code=CRIX.UPBIT.KRW-' + coinName +'&count='+str(cnt)
    else :
        basic_url = addr + type + '/' + '?code=CRIX.UPBIT.KRW-' + coinName +'&count='+str(cnt)

    return basic_url

basic_url = get_coin_data_url(coinName,type,scale)
url = basic_url

def get_upbit_data(url, last_date, to_date) :
    global df
    global index

    fail2GetData = False
    response = ''
    while True:
        try :
            response = requests.get(url)
        except Exception as e:
            time.sleep(20)
            continue
        if str(response) == '<Response [200]>':
            break
        time.sleep(10)
    if ( fail2GetData )  :
        exit()

    data = response.json()

    if index == 0:
        df = pd.DataFrame(data)
        date = ''
        date = data[len(data)-1]['candleDateTime']
        index +=1

    else:
        df_for = pd.DataFrame(data)
        df = df.append(df_for,ignore_index=True)
        date = ''
        date = data[len(data)-1]['candleDateTime']
    return date




In [18]:

while (1) :
    last_date = get_upbit_data(url, last_date, to_date)
    tmp1 = last_date.split('T')
    if tmp1[0]  < to_date :
        break
    tmp2 = tmp1[1].split('+')
    target_date = tmp1[0] + ' ' + tmp2[0]
    url = basic_url + '&to=' + target_date    #to=2019-11-27 04:01:00
    time.sleep(2)

idx_todate = df.index[(df.candleDateTimeKst == '{}T00:00:00+09:00'.format(to_date))].tolist()[0]+1
idx_todate

df = df.drop(df.index[idx_todate:])

In [19]:
df['Date'] = 0
df['hour'] = 0
df['minute'] = 0

In [20]:
for i in range(len(df)):
    df['Date'] [i] = df['candleDateTimeKst'][i].split('T')
    df['hour'] [i] = df['Date'][i][1].split('+')[0]
    df['minute'] [i] = df['hour'][i].split(':')[1]
    df['hour'] [i] = df['hour'][i].split(':')[0]
    df['Date'] [i] = df['Date'][i][0]


In [21]:
drops= ['Date','unit','candleAccTradePrice','openingPrice','code','candleDateTime','timestamp','candleDateTimeKst']
df=df.drop(drops,axis=1)
df=df[['hour','minute','tradePrice','highPrice','lowPrice','candleAccTradeVolume']]
df.head()

,hour,minute,tradePrice,highPrice,lowPrice,candleAccTradeVolume
0,03,0,38114000.0,38402000.0,38032000.0,84.610371
1,02,30,38400000.0,38463000.0,38316000.0,43.053330
2,02,0,38375000.0,38448000.0,38229000.0,72.150857
3,01,30,38347000.0,38510000.0,38206000.0,119.634094
4,01,0,38478000.0,38782000.0,38399000.0,163.076826


In [22]:
ups, downs = df['tradePrice'].loc[::-1].diff() ,df['tradePrice'].loc[::-1].diff()
ups[ups<0] =0
downs[downs>0] = 0
period = 14 
au = ups.abs().ewm(com = period-1, min_periods = period).mean() 
ad = downs.abs().ewm(com = period-1, min_periods = period).mean() 
RS = au/ad 
RSI = pd.Series(100 - (100/(1+RS))) 
RSI = RSI.loc[::-1]
print(RSI)
df['RSI'] = RSI

0      45.857695
1      52.795071
2      52.208223
3      51.582220
4      54.694627
         ...    
338          NaN
339          NaN
340          NaN
341          NaN
342          NaN
Name: tradePrice, Length: 343, dtype: float64


In [23]:
# CCI 오예
N = 20
D = 0
m = 0
for i in range(3):
    D = 0
    m = 0
    sm= 0

    M = ((df['tradePrice'][i] + df['lowPrice'][i] + df['highPrice'][i]) / 3)

    for j in range(N):
        jM = (df['tradePrice'][i+j] + df['lowPrice'][i+j] + df['highPrice'][i+j]) / 3
        sm +=jM
    sm = (sm/N)

    for k in range(N):
        kM = (df['tradePrice'][i+k] + df['lowPrice'][i+k] + df['highPrice'][i+k]) / 3
        D += abs(sm-kM)
    D = D/N
    print('D : {}  // M : {} // m : {}'.format(D,M,sm))

    CCI = (M - sm) / (D * 0.015)

    print(CCI)


D : 217716.66666666715  // M : 38182666.666666664 // m : 38361049.99999999
-54.6224705912357
D : 216738.33333333433  // M : 38393000.0 // m : 38379783.33333333
4.06532198324295
D : 225603.33333333433  // M : 38350666.666666664 // m : 38389633.33333333
-11.514802283259412


In [24]:
s_data, l_data = df['tradePrice'].loc[::-1],df['tradePrice'].loc[::-1]
s_data
l_data

342    40830000.0
341    40838000.0
340    40938000.0
339    40870000.0
338    41017000.0
          ...    
4      38478000.0
3      38347000.0
2      38375000.0
1      38400000.0
0      38114000.0
Name: tradePrice, Length: 343, dtype: float64

In [25]:
# MACD
period = 12
s_data, l_data = df['tradePrice'].loc[::-1],df['tradePrice'].loc[::-1]
# s_data, l_data = df['tradePrice'].diff() ,df['tradePrice'].diff()
a12 = s_data.abs().ewm(com = period-1, min_periods = period).mean() 
a12.head(15)


342             NaN
341             NaN
340             NaN
339             NaN
338             NaN
337             NaN
336             NaN
335             NaN
334             NaN
333             NaN
332             NaN
331    4.138548e+07
330    4.144915e+07
329    4.148463e+07
328    4.150365e+07
Name: tradePrice, dtype: float64

In [26]:
s_data = df.loc[::-1]
s_data

,hour,minute,tradePrice,highPrice,lowPrice,candleAccTradeVolume,RSI
342,00,0,40830000.0,40980000.0,40600000.0,345.947578,NaN
341,00,30,40838000.0,40960000.0,40755000.0,149.095600,NaN
340,01,0,40938000.0,40950000.0,40700000.0,163.617955,NaN
339,01,30,40870000.0,40938000.0,40759000.0,71.849834,NaN
338,02,0,41017000.0,41040000.0,40788000.0,84.941353,NaN
...,...,...,...,...,...,...,...
4,01,0,38478000.0,38782000.0,38399000.0,163.076826,54.694627
3,01,30,38347000.0,38510000.0,38206000.0,119.634094,51.582220
2,02,0,38375000.0,38448000.0,38229000.0,72.150857,52.208223
1,02,30,38400000.0,38463000.0,38316000.0,43.053330,52.795071


In [27]:
def fnMACD(m_Df, m_NumFast=12, m_NumSlow=26, m_NumSignal=9):
    m_Df['EMAFast'] = m_Df['tradePrice'].ewm( span = m_NumFast, min_periods = m_NumFast - 1).mean()
    m_Df['EMASlow'] = m_Df['tradePrice'].ewm( span = m_NumSlow, min_periods = m_NumSlow - 1).mean()
    m_Df['MACD'] = m_Df['EMAFast'] - m_Df['EMASlow']
    m_Df['MACDSignal'] = m_Df['MACD'].ewm( span = m_NumSignal, min_periods = m_NumSignal-1).mean()
    m_Df['MACDDiff'] = m_Df['MACD'] - m_Df['MACDSignal']
    return m_Df

s_data = fnMACD(s_data)
s_data

,hour,minute,tradePrice,highPrice,lowPrice,candleAccTradeVolume,RSI,EMAFast,EMASlow,MACD,MACDSignal,MACDDiff
342,00,0,40830000.0,40980000.0,40600000.0,345.947578,NaN,NaN,NaN,NaN,NaN,NaN
341,00,30,40838000.0,40960000.0,40755000.0,149.095600,NaN,NaN,NaN,NaN,NaN,NaN
340,01,0,40938000.0,40950000.0,40700000.0,163.617955,NaN,NaN,NaN,NaN,NaN,NaN
339,01,30,40870000.0,40938000.0,40759000.0,71.849834,NaN,NaN,NaN,NaN,NaN,NaN
338,02,0,41017000.0,41040000.0,40788000.0,84.941353,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
4,01,0,38478000.0,38782000.0,38399000.0,163.076826,54.694627,3.848404e+07,3.828924e+07,194803.439151,221538.436455,-26734.997304
3,01,30,38347000.0,38510000.0,38206000.0,119.634094,51.582220,3.846296e+07,3.829352e+07,169441.225652,211118.994294,-41677.768643
2,02,0,38375000.0,38448000.0,38229000.0,72.150857,52.208223,3.844943e+07,3.829956e+07,149873.209654,198869.837366,-48996.627712
1,02,30,38400000.0,38463000.0,38316000.0,43.053330,52.795071,3.844182e+07,3.830700e+07,134828.488179,186061.567529,-51233.079350


In [28]:
s_data.drop(['EMAFast','EMASlow'],axis=1)
s_data

,hour,minute,tradePrice,highPrice,lowPrice,candleAccTradeVolume,RSI,EMAFast,EMASlow,MACD,MACDSignal,MACDDiff
342,00,0,40830000.0,40980000.0,40600000.0,345.947578,NaN,NaN,NaN,NaN,NaN,NaN
341,00,30,40838000.0,40960000.0,40755000.0,149.095600,NaN,NaN,NaN,NaN,NaN,NaN
340,01,0,40938000.0,40950000.0,40700000.0,163.617955,NaN,NaN,NaN,NaN,NaN,NaN
339,01,30,40870000.0,40938000.0,40759000.0,71.849834,NaN,NaN,NaN,NaN,NaN,NaN
338,02,0,41017000.0,41040000.0,40788000.0,84.941353,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
4,01,0,38478000.0,38782000.0,38399000.0,163.076826,54.694627,3.848404e+07,3.828924e+07,194803.439151,221538.436455,-26734.997304
3,01,30,38347000.0,38510000.0,38206000.0,119.634094,51.582220,3.846296e+07,3.829352e+07,169441.225652,211118.994294,-41677.768643
2,02,0,38375000.0,38448000.0,38229000.0,72.150857,52.208223,3.844943e+07,3.829956e+07,149873.209654,198869.837366,-48996.627712
1,02,30,38400000.0,38463000.0,38316000.0,43.053330,52.795071,3.844182e+07,3.830700e+07,134828.488179,186061.567529,-51233.079350


In [29]:
s_data = s_data.loc[::-1]

In [30]:
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.style.use('ggplot')
mpl.rcParams['axes.unicode_minus'] = False
from sklearn.metrics import make_scorer


In [31]:

def rmsle(predicted_values, actual_values, convertExp=True):

    if convertExp:
        predicted_values = np.exp(predicted_values),
        actual_values = np.exp(actual_values)
        
    # 넘파이로 배열 형태로 바꿔준다.
    predicted_values = np.array(predicted_values)
    actual_values = np.array(actual_values)
    
    # 예측값과 실제 값에 1을 더하고 로그를 씌워준다.
    # 값이 0일 수도 있어서 로그를 취했을 때 마이너스 무한대가 될 수도 있기 때문에 1을 더해 줌
    # 로그를 씌워주는 것은 정규분포로 만들어주기 위해
    log_predict = np.log(predicted_values + 1)
    log_actual = np.log(actual_values + 1)
    
    # 위에서 계산한 예측값에서 실제값을 빼주고 제곱을 해준다.
    difference = log_predict - log_actual
    difference = np.square(difference)
    
    # 평균을 낸다.
    mean_difference = difference.mean()
    
    # 다시 루트를 씌운다.
    score = np.sqrt(mean_difference)
    
    return score

In [32]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
import warnings
pd.options.mode.chained_assignment = None
warnings.filterwarnings("ignore", category=DeprecationWarning)

# 선형회귀 모델을 초기화
lModel = LinearRegression()

# 모델을 학습시킨다.
y_train_log = np.log1p(df)
lModel.fit(X_train, y_train_log)

# 예측하고 정확도를 평가한다.
preds = lModel.predict(X_train)
print ("RMSLE Value For Linear Regression: ",
       rmsle(np.exp(y_train_log),np.exp(preds), False))

TypeError: loop of ufunc does not support argument 0 of type str which has no callable log1p method